In [1]:
#Analysing the scraped data
import pandas as pd

In [2]:
#Read the data
df = pd.read_csv('data/tweets.csv')
df.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,entities,favorited,retweeted,filter_level,lang,timestamp_ms,possibly_sensitive,extended_entities,display_text_range,extended_tweet
0,Sat Jun 06 19:05:06 +0000 2020,1269344584729006081,1269344584729006081,RT @joseluisescriva: A pesar de la derrota tan...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,es,1591470306938,NaN,NaN,NaN,NaN
1,Sat Jun 06 19:05:07 +0000 2020,1269344585014218753,1269344585014218753,RT @SecAzar: Great to be with President Trump ...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1591470307006,NaN,NaN,NaN,NaN
2,Sat Jun 06 19:05:07 +0000 2020,1269344585089789953,1269344585089789953,RT @TheAtlantic: “The lives of disproportionat...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1591470307024,NaN,NaN,NaN,NaN
3,Sat Jun 06 19:05:07 +0000 2020,1269344585320402946,1269344585320402946,RT @SaunderHart: à esquerda tudo é permitido.....,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,pt,1591470307079,NaN,NaN,NaN,NaN
4,Sat Jun 06 19:05:07 +0000 2020,1269344585563635716,1269344585563635716,RT @Helen504: I lost my brother in law from co...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,NaN,NaN,NaN,NaN,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1591470307137,NaN,NaN,NaN,NaN


In [3]:
#Columns
df.columns

Index(['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'retweeted_status', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status', 'quoted_status_permalink',
       'is_quote_status', 'quote_count', 'reply_count', 'retweet_count',
       'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level',
       'lang', 'timestamp_ms', 'possibly_sensitive', 'extended_entities',
       'display_text_range', 'extended_tweet'],
      dtype='object')

In [4]:
#Description
df.describe()

,id,id_str,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,contributors,quoted_status_id,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,timestamp_ms
count,1.000000e+03,1.000000e+03,9.700000e+01,9.700000e+01,1.030000e+02,1.030000e+02,0.0,2.720000e+02,2.720000e+02,1000.0,1000.0,1000.0,1000.0,1.000000e+03
mean,1.269345e+18,1.269345e+18,1.269281e+18,1.269281e+18,3.361108e+17,3.361108e+17,NaN,1.268793e+18,1.268793e+18,0.0,0.0,0.0,0.0,1.591470e+12
std,3.201333e+10,3.201333e+10,2.043401e+14,2.043401e+14,5.122106e+17,5.122106e+17,NaN,2.638990e+15,2.638990e+15,0.0,0.0,0.0,0.0,7.632574e+03
min,1.269345e+18,1.269345e+18,1.267538e+18,1.267538e+18,9.390910e+05,9.390910e+05,NaN,1.239253e+18,1.239253e+18,0.0,0.0,0.0,0.0,1.591470e+12
25%,1.269345e+18,1.269345e+18,1.269305e+18,1.269305e+18,9.620872e+07,9.620872e+07,NaN,1.269030e+18,1.269030e+18,0.0,0.0,0.0,0.0,1.591470e+12
50%,1.269345e+18,1.269345e+18,1.269337e+18,1.269337e+18,1.000699e+09,1.000699e+09,NaN,1.269169e+18,1.269169e+18,0.0,0.0,0.0,0.0,1.591470e+12
75%,1.269345e+18,1.269345e+18,1.269343e+18,1.269343e+18,9.104223e+17,9.104223e+17,NaN,1.269291e+18,1.269291e+18,0.0,0.0,0.0,0.0,1.591470e+12
max,1.269345e+18,1.269345e+18,1.269345e+18,1.269345e+18,1.266386e+18,1.266386e+18,NaN,1.269344e+18,1.269344e+18,0.0,0.0,0.0,0.0,1.591470e+12


In [5]:
#Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 36 columns):
created_at                   1000 non-null object
id                           1000 non-null int64
id_str                       1000 non-null int64
text                         1000 non-null object
source                       1000 non-null object
truncated                    1000 non-null bool
in_reply_to_status_id        97 non-null float64
in_reply_to_status_id_str    97 non-null float64
in_reply_to_user_id          103 non-null float64
in_reply_to_user_id_str      103 non-null float64
in_reply_to_screen_name      103 non-null object
user                         1000 non-null object
geo                          1 non-null object
coordinates                  1 non-null object
place                        15 non-null object
contributors                 0 non-null float64
retweeted_status             705 non-null object
quoted_status_id             272 non-null float64
quoted_statu

In [6]:
df['possibly_sensitive'].describe()

count       197
unique        2
top       False
freq        191
Name: possibly_sensitive, dtype: object

In [7]:
df['lang'].describe()

count     1000
unique      21
top         en
freq       567
Name: lang, dtype: object

In [10]:
df_en = df.loc[df['lang'] == 'en']
df_en.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,entities,favorited,retweeted,filter_level,lang,timestamp_ms,possibly_sensitive,extended_entities,display_text_range,extended_tweet
1,Sat Jun 06 19:05:07 +0000 2020,1269344585014218753,1269344585014218753,RT @SecAzar: Great to be with President Trump ...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1591470307006,NaN,NaN,NaN,NaN
2,Sat Jun 06 19:05:07 +0000 2020,1269344585089789953,1269344585089789953,RT @TheAtlantic: “The lives of disproportionat...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1591470307024,NaN,NaN,NaN,NaN
4,Sat Jun 06 19:05:07 +0000 2020,1269344585563635716,1269344585563635716,RT @Helen504: I lost my brother in law from co...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,NaN,NaN,NaN,NaN,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1591470307137,NaN,NaN,NaN,NaN
5,Sat Jun 06 19:05:07 +0000 2020,1269344585555247114,1269344585555247114,IBM’s OpenPandemics Tool Lets You Contribute t...,"<a href=""https://www.hootsuite.com"" rel=""nofol...",False,NaN,NaN,NaN,NaN,...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,1591470307135,False,NaN,NaN,NaN
6,Sat Jun 06 19:05:07 +0000 2020,1269344585748078592,1269344585748078592,RT @peterelkind: Trump’s photo op at a Maine f...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1591470307181,NaN,NaN,NaN,NaN


In [11]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 567 entries, 1 to 999
Data columns (total 36 columns):
created_at                   567 non-null object
id                           567 non-null int64
id_str                       567 non-null int64
text                         567 non-null object
source                       567 non-null object
truncated                    567 non-null bool
in_reply_to_status_id        55 non-null float64
in_reply_to_status_id_str    55 non-null float64
in_reply_to_user_id          57 non-null float64
in_reply_to_user_id_str      57 non-null float64
in_reply_to_screen_name      57 non-null object
user                         567 non-null object
geo                          1 non-null object
coordinates                  1 non-null object
place                        9 non-null object
contributors                 0 non-null float64
retweeted_status             414 non-null object
quoted_status_id             172 non-null float64
quoted_status_id_str    

In [12]:
#Extracting just the basic info from data (english tweets)
data = df_en = df.loc[df['lang'] == 'en'][['created_at', 'text', 'possibly_sensitive']]
data.head()

,created_at,text,possibly_sensitive
1,Sat Jun 06 19:05:07 +0000 2020,RT @SecAzar: Great to be with President Trump ...,NaN
2,Sat Jun 06 19:05:07 +0000 2020,RT @TheAtlantic: “The lives of disproportionat...,NaN
4,Sat Jun 06 19:05:07 +0000 2020,RT @Helen504: I lost my brother in law from co...,NaN
5,Sat Jun 06 19:05:07 +0000 2020,IBM’s OpenPandemics Tool Lets You Contribute t...,False
6,Sat Jun 06 19:05:07 +0000 2020,RT @peterelkind: Trump’s photo op at a Maine f...,NaN
